In [14]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
}


# Slette gamle brukarar
Først les eg inn dei som er mest aktuelle å slette i første omgang:


In [15]:
andre = pd.read_csv("andre.csv")
utan_sis = pd.read_csv("utan_sis.csv")


## Finne sist pålogging


### Forsøk 1 (ikkje optimalt)
Neste steg er tabellen `enrollments`; her får eg feltet `last_activity_at` som i følge dokumentasjonen er 

> The last activity time of the user for the enrollment. This time stamp updates when a user navigates through the course using the Course Navigation menu, replies to a discussion or submits an assignment or quiz, and when they access course files and pages (does not include mobile interactions or group participation).

Det eg får ut er siste gang ein brukar var inne i eit gitt emne. Så eg må sjå på *alle* slike oppføringar for ein brukar, og så plukke ut den yngste. NB! Eg har henta data med skriptet "les_CD2_tabell.ipynb", men eg har berre lagra `user_id` og `course_id`. Dokumentasjonen tilrår å sjekke `course_section_id` og `role_id` også, for ein brukar kan jo ha vore logga på som student i eit emne og litt seinare som tilsett (eller studentassistent) i emnet. Men eg trur risikoen for dette er liten (det er jo eksterne faglærarar og kursdeltakarar dette).

Eg vil helst gå gjennom lista `utan_sis` og sjekke siste pålogging for alle der (basert på `id`). Alle som har blank eller tidlegare enn 2024 kan eg sikkert slette?

### Forsøk 2: 
Eg prøver så CD2-tabellen "users"; den har feltet `last_logged_out`. Eg har tatt eit uttrekk av data etter 1. mai 2025: 

In [4]:
utlogga = pd.read_csv("brukarar_last_logged_out.csv")

In [ ]:
utlogga.head()

In [ ]:
andre.head()

In [ ]:
utan_sis.head()

No er `key.id` og `id` felles nøkkel i alle tabellane, så eg kan gå gjennom alle i ei av `utan_sis` og vise (om dei finst i `utlogga`) dato for sist utlogga: 

In [8]:
utlogga_utan_sis = pd.merge(utlogga, utan_sis, left_on='key.id', right_on='id', how='right')

In [ ]:
utlogga_utan_sis[utlogga_utan_sis['value.last_logged_out'].notnull()]

In [10]:
utlogga_andre = pd.merge(utlogga, andre, left_on='key.id', right_on='id', how='right')

In [ ]:
utlogga_andre[utlogga_andre['value.last_logged_out'].notnull()]

In [12]:
eldre = utlogga_andre[utlogga_andre['value.last_logged_out'] < '2024-01-01']

In [ ]:
eldre

Konklusjon: det ser ut som at det er veldig vanskeleg å finne noko her. Så det kan hende eg må gå tilbakee til å sjekke siste aktivitet i kvart emne :-(

### Forsøk 3 (forsøk 1 ein gang til)

Eg må altså ta utgangspunkt i listene `utan_sis` og  `andre`, og så finne alle enrollments for dei personane. `enrollments` kan eg finne via CD2.

I fila `enrollments.csv` har eg ei liste over person/emne/siste aktivitet, og då kan eg koble denne mot `utan_sis` og `andre`.